In [1]:
# importing libraries 
import pandas as pd
import torch
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from torch.utils.data import ConcatDataset
import os
from datasets import Dataset
# import packages (remember to import the local_transoformers package in your folder correctly!)
from local_transformers_bioClinical_BERT.src.transformers import TrainingArguments
import tensorflow as tf
from local_transformers_bioClinical_BERT.src.transformers.models.biogpt.tokenization_biogpt import BioGptTokenizer # reads the local_transformers folder instead of the actual transformers package
from local_transformers_bioClinical_BERT.src.transformers.models.CustombioClinicalBERT import CustomBioClinicalBertForCombinedLearning # reads the local_transformers folder instead of the actual transformers package
from local_transformers_bioClinical_BERT.src.transformers.CustomComputeLoss import CustomTrainer # reads the local_transformers folder instead of the actual transformers package
import os
import pickle
from local_transformers_bioClinical_BERT.src.transformers.trainer_callback import TrainerCallback
import numpy as np
from xgboost import XGBRegressor, XGBClassifier
from numpy import mean, std
from sklearn.metrics import roc_auc_score, average_precision_score, precision_score, recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
from local_transformers_bioClinical_BERT.src.transformers.data.data_collator import DataCollatorForLanguageModeling
from local_transformers_bioClinical_BERT.src.transformers.models.auto.tokenization_auto import AutoTokenizer
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

if tf.test.is_gpu_available():
    print('GPU is available')
else:
    print('GPU is not available, running on CPU')

# Reading data
df=pd.read_csv("df.csv")
df["death_in_30"] = df["death_in_30"].astype(int)
df["DVT"] = df["DVT"].astype(int)
df["PE"] = df["PE"].astype(int)
df["post_aki_status"] = df["post_aki_status"].astype(int)
df["PNA"] = df["PNA"].astype(int)
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = CustomBioClinicalBertForCombinedLearning.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",output_hidden_states=True,weights=None,lambda_constant=2
) #NOTE: Change lambda constant to the appropriate one


# preparing the clinical notes for training and fine-tuning 
# this function here helps set up and prepare the data for fine-tuning, including those of the additional labels. 
def tokenize_and_prepare(batch, text_col, outcome_col, task_id):
    tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    encodings = tokenizer(batch[text_col], padding='max_length', truncation=True, max_length=164)
    additional_labels = torch.tensor(batch[outcome_col])  # make sure additional_labels are tensors
    task_ids = torch.tensor([task_id] * len(additional_labels))
    return {
        "input_ids": encodings["input_ids"], 
        "attention_mask": encodings["attention_mask"], 
        "additional_labels": additional_labels, 
        "labels": encodings["input_ids"], 
        "task_ids": task_ids
    }


# helping prepare the data per task given the multi-label nature of foundational models. 
def prepare_data_per_task(df, text_col, outcome_col, task_id):
    df = df.dropna(subset=[outcome_col]).reset_index(drop=True)
    df[outcome_col]=df[outcome_col].astype(int)
    texts = list(df[text_col])
    outcomes = list(df[outcome_col])
    data = {text_col: texts, outcome_col: outcomes}
    dataset = Dataset.from_dict(data)
    dataset = dataset.map(lambda examples: tokenize_and_prepare(examples, text_col, outcome_col, task_id), batched=True, batch_size=128)
    dataset.set_format("torch", columns=["input_ids", "attention_mask", "additional_labels", "labels", "task_ids"])
    return dataset


# By default Trainer actually shuffles the order of the data, but we can use this as well if we wish to
# class ShuffledDataset(torch.utils.data.Dataset):
#     def __init__(self, dataset):
#         self.dataset = dataset
#         indices = torch.randperm(len(dataset))
#         self.shuffled_data = [dataset[i] for i in indices]

#     def __getitem__(self, idx):
#         return self.shuffled_data[idx]

#     def __len__(self):
#         return len(self.dataset)

# stacking the data of different labels on top of one another so we can perform MTL fine-tuning
def stack_data(df, tasks):
    datasets = []
    for i, task in enumerate(tasks):
        dataset = prepare_data_per_task(df, "AN_PROC_NAME", task, i)
        datasets.append(dataset)
    combined_dataset = ConcatDataset(datasets)
    # combined_dataset=ShuffledDataset(combined_dataset)
    return(combined_dataset)

2023-11-04 01:26:45.949306: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-04 01:26:46.412351: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-11-04 01:26:46.412393: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-11-04 01:26:46.412397: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing CustomBioClinicalBertForCombinedLearning: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing CustomBioClinicalBertForCombinedLearning from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomBioClinicalBertForCombinedLearning from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CustomBioClinicalBertForCombinedLearning were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['auxiliary.1.bias', 'auxiliary.0.bias', 'auxiliary.5.bias', 'auxiliary.5.weight', 'auxiliary.0.weight', 'auxili

In [2]:
# this finetunes our model
def get_model(name, train_dataset,val_dataset):
    tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    model = CustomBioClinicalBertForCombinedLearning.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",output_hidden_states=True,weights=None,lambda_constant=2,num_tasks=6) #NOTE: Change lambda constant to the appropriate one
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=6,
        per_device_train_batch_size=48,
        per_device_eval_batch_size=48,
        warmup_steps=1500, 
        weight_decay=0.001,
        learning_rate=0.00001, 
        logging_dir="./logs",
        logging_steps=1000,
        save_strategy="epoch")
    # Define the data collator for masked language modeling
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
    # Instantiate the Trainer
    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer)
    trainer.train()
    trainer.model.save_pretrained(name)
    trainer.tokenizer.save_pretrained(name)

In [3]:
# gets the text representation from the updated specified model
def get_bioclinicalBERT_updated(texts, name):
    notes = list(texts)
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = CustomBioClinicalBertForCombinedLearning.from_pretrained(name, output_hidden_states=True).to(device)

    # Check if CUDA is available and if so, move the model to GPU
    if torch.cuda.is_available():
        model = model.to("cuda")

    # Tokenize texts and create input IDs and attention masks
    input_ids, attention_masks = [], []
    for text in notes:
        encoded_input = tokenizer.encode_plus(text, max_length=128, padding='max_length', truncation=True,
                                              return_attention_mask=True, return_tensors='pt')  # change max_length if needed [NOTE: max_length may differ across task]
        input_ids.append(encoded_input['input_ids'])
        attention_masks.append(encoded_input['attention_mask'])

    # Convert lists to tensors
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    # If CUDA is available, move the tensors to GPU
    if torch.cuda.is_available():
        input_ids = input_ids.to("cuda")
        attention_masks = attention_masks.to("cuda")

    # Batch size for processing
    batch_size = 32
    # Initialize an empty list to store embeddings
    embeddings = []
    # Process input IDs and attention masks in batches
    for i in tqdm(range(0, len(input_ids), batch_size)):
        input_ids_batch = input_ids[i:i+batch_size]
        attention_masks_batch = attention_masks[i:i+batch_size]

        with torch.no_grad():
            # Get the model's output
            output = model(input_ids=input_ids_batch, attention_mask=attention_masks_batch)

        # Retrieve the last_hidden_state from the output
        last_hidden_state = output.hidden_states[-1]

        # Optionally, you can obtain the pooled_output by averaging or max-pooling the last_hidden_state
        pooled_output = torch.mean(last_hidden_state, dim=1)

        # If CUDA is available, move the embeddings back to CPU
        if torch.cuda.is_available():
            pooled_output = pooled_output.to("cpu")

        # Add the embeddings to the list
        embeddings.extend(pooled_output.numpy())

    embeddings = np.array(embeddings)
    return embeddings


## Default implementation of text representation + XGBoost

In [4]:

def rounding(take_mean, take_std):
    return(round(take_mean, 2), round(take_std,2))

def K_fold_val(outcome_col, df):
    # Initialize the StratifiedKFold class
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Initialize lists to hold scores        
    auroc_scores, auprc_scores,accuracy_scores, recall_scores,precision_scores, specificity_scores,f_scores = [],[],[],[],[],[],[]

    # changed parameters here because initial parameters gave underwhelming results
    # so i explored alternative hyperparameters to see if model improves
    param_grid = {
        'learning_rate': [0.1,0.15,0.3],
        'max_depth': [4,5,6,7,8],
        'min_child_weight':[1,2,4]
        }
    i=1
    # Perform 5-fold cross validation
    for train_index, test_index in tqdm(skf.split(df, df["PE"])):
        train, test = df.iloc[train_index], df.iloc[test_index]
        name=f"fold_{i}"            
        i=i+1
        if os.path.isdir(name):
            print("passed")
        else:
            # train val split
            train_new=((train.sample(frac=7/8,random_state=42)))
            val_new = train[~train.index.isin(train_new.index)]
            train_new=(train_new.reset_index(drop=True))
            val_new=(val_new.reset_index(drop=True))
            tasks=["death_in_30","DVT","PE","PNA","post_aki_status","postop_del"]
            train_new=stack_data(train_new, tasks)
            val_new=stack_data(val_new, tasks)
            get_model(name, train_new,val_new)
        if os.path.isfile(f"{name}.pickle"):
            with open(f"{name}.pickle", 'rb') as file:
                data = pickle.load(file)
                X_train=data[0]
                X_test=data[1]
        else:
            X_train=get_bioclinicalBERT_updated(train["AN_PROC_NAME"], name)
            X_test = get_bioclinicalBERT_updated(test["AN_PROC_NAME"], name)
                    
            with open(f"{name}.pickle", 'wb') as f:
                pickle.dump([X_train,X_test], f)
        
        # Prepare training and testing data

        y_train = train[outcome_col]
        y_test = test[outcome_col]

        # dropping potential NA rows
        if y_train.isna().any() or y_test.isna().any(): 
            na_indices = y_train.isna()  
            X_train = (X_train[~na_indices])
            y_train = (((y_train.dropna())).astype(int)).reset_index(drop=True)
            na_indices = y_test.isna()  
            X_test = X_test[~na_indices]  
            y_test = ((y_test.dropna()).astype(int)).reset_index(drop=True)
        else:
            pass
        # running XGBoost
        model = xgb.XGBClassifier(random_state=42,tree_method='gpu_hist')
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='average_precision',verbose=2)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict_proba(X_test)[:, 1]
        y_pred_class = best_model.predict(X_test)

        # Compute the AUROC and AUPRC
        auroc = roc_auc_score(y_test, y_pred)
        auprc = average_precision_score(y_test, y_pred)
        accuracy=accuracy_score(y_test, y_pred_class)
        recall = recall_score(y_test, y_pred_class)
        precision = precision_score(y_test, y_pred_class)
        # For specificity (True Negative Rate)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred_class).ravel()
        specificity = tn / (tn + fp)
        f_score=f1_score(y_test, y_pred_class)

        # Save the scores
        auroc_scores.append(auroc)
        auprc_scores.append(auprc)
        accuracy_scores.append(accuracy)
        recall_scores.append(recall)
        precision_scores.append(precision)
        specificity_scores.append(specificity)
        f_scores.append(f_score)

        with open(f"update_{outcome_col}.pickle", 'wb') as f:
            pickle.dump({"auprc":auprc_scores, "auroc":auroc_scores,"accuracy":accuracy_scores,
                         "precision":precision_scores,"recall":recall_scores, 
                         "specificity":specificity_scores,
                         "f1":f_scores},f)

    
    # Now you can calculate the mean and standard deviation
    mean_auroc,std_auroc = rounding(np.mean(auroc_scores), np.std(auroc_scores))
    mean_auprc, std_auprc = rounding(np.mean(auprc_scores), np.std(auprc_scores))
    mean_accuracy, std_accuracy=rounding(np.mean(accuracy_scores), np.std(accuracy_scores))
    mean_precision, std_precision=rounding(np.mean(precision_scores), np.std(precision_scores))
    mean_recall, std_recall=rounding(np.mean(recall_scores), np.std(recall_scores))
    mean_specificity, std_specificity=rounding(np.mean(specificity_scores), np.std(specificity_scores))
    mean_f1,std_f1=rounding(np.mean(f_scores), np.std(f_scores))  
    
    print_statement=f'''
    metrics: Mean AUROC: {mean_auroc}, SD AUROC: {std_auroc}, 
    \n Mean AUPRC: {mean_auprc}, SD AUPRC: {std_auprc}, 
    \n Mean ACC:{mean_accuracy}, SD ACC:{std_accuracy}, 
    \n Mean precision:{mean_precision}, std precision:{std_precision}, 
    \n mean recall:{mean_recall}, std_recall:{std_recall},
    \n mean specificity:{mean_specificity}, std specificity:{std_specificity}, 
    \n mean f1:{mean_f1}, std f1:{std_f1} 
    '''
    return(print_statement)


## Text representation + Logistic Regression

In [5]:
def K_fold_val_log_reg(outcome_col, df):
    # Initialize the StratifiedKFold class
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Initialize lists to hold scores
    auroc_scores = []
    auprc_scores = []

    # changed parameters here because initial parameters gave underwhelming results
    # so i explored alternative hyperparameters to see if model improves
    param_grid = {
    'C': [0.01, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['lbfgs', 'newton-cholesky']
    }
    i=1
    # Perform 5-fold cross validation
    for train_index, test_index in tqdm(skf.split(df, df["PE"])):
        train, test = df.iloc[train_index], df.iloc[test_index]
        name=f"fold_{i}"            
        i=i+1
        if os.path.isdir(name):
            print("passed")
        else:
            train_new=((train.sample(frac=7/8,random_state=42)))
            val_new = train[~train.index.isin(train_new.index)]
            train_new=(train_new.reset_index(drop=True))
            val_new=(val_new.reset_index(drop=True))
            tasks=["death_in_30","DVT","PE","PNA","post_aki_status","postop_del"]
            train_new=stack_data(train_new, tasks)
            val_new=stack_data(val_new, tasks)
            get_model(name, train_new,val_new)
        if os.path.isfile(f"{name}.pickle"):
            with open(f"{name}.pickle", 'rb') as file:
                data = pickle.load(file)
                X_train=data[0]
                X_test=data[1]
        else:
            X_train=get_bioclinicalBERT_updated(train["AN_PROC_NAME"], name)
            X_test = get_bioclinicalBERT_updated(test["AN_PROC_NAME"], name)
                    
            with open(f"{name}.pickle", 'wb') as f:
                pickle.dump([X_train,X_test], f)
        # Prepare training and testing data

        y_train = train[outcome_col]
        y_test = test[outcome_col]

        if y_train.isna().any() or y_test.isna().any(): 
            na_indices = y_train.isna()  
            X_train = (X_train[~na_indices])
            y_train = (((y_train.dropna())).astype(int)).reset_index(drop=True)
            na_indices = y_test.isna()  
            X_test = X_test[~na_indices]  
            y_test = ((y_test.dropna()).astype(int)).reset_index(drop=True)
        else:
            pass
        
        model = LogisticRegression(random_state=42,max_iter=10000)
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='average_precision',verbose=2)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict_proba(X_test)[:, 1]

        # Compute the AUROC and AUPRC
        auroc = roc_auc_score(y_test, y_pred)
        auprc = average_precision_score(y_test, y_pred)

        # Save the scores
        auroc_scores.append(auroc)
        auprc_scores.append(auprc)
        with open(f"update_{outcome_col}_logreg.pickle", 'wb') as f:
            pickle.dump({"auprc":auprc_scores, "auroc":auroc_scores}, f)

    # Now you can calculate the mean and standard deviation
    mean_auroc = np.mean(auroc_scores)
    std_auroc = np.std(auroc_scores)
    mean_auprc = np.mean(auprc_scores)
    std_auprc = np.std(auprc_scores)
    
    return(f"metrics: Mean AUROC: {mean_auroc}, SD AUROC: {std_auroc}, Mean AUPRC: {mean_auprc}, SD AUPRC: {std_auprc}")


## Text representation + Random Forest

In [6]:
def K_fold_val_rf(outcome_col, df):
    # Initialize the StratifiedKFold class
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Initialize lists to hold scores
    auroc_scores = []
    auprc_scores = []

    # changed parameters here because initial parameters gave underwhelming results
    # so i explored alternative hyperparameters to see if model improves
    param_grid = {
        'max_depth': [4, None],  # maximum depth of the tree
        'min_samples_leaf': [1, 3],  # minimum number of samples required to be at a leaf node
        }
    i=1
    # Perform 5-fold cross validation
    for train_index, test_index in tqdm(skf.split(df, df["PE"])):
        train, test = df.iloc[train_index], df.iloc[test_index]
        name=f"fold_{i}"            
        i=i+1
        if os.path.isdir(name):
            print("passed")
        else:
            train_new=((train.sample(frac=7/8,random_state=42)))
            val_new = train[~train.index.isin(train_new.index)]
            train_new=(train_new.reset_index(drop=True))
            val_new=(val_new.reset_index(drop=True))
            tasks=["death_in_30","DVT","PE","PNA","post_aki_status","postop_del"]
            train_new=stack_data(train_new, tasks)
            val_new=stack_data(val_new, tasks)
            get_model(name, train_new,val_new)
        if os.path.isfile(f"{name}.pickle"):
            with open(f"{name}.pickle", 'rb') as file:
                data = pickle.load(file)
                X_train=data[0]
                X_test=data[1]
        else:
            X_train=get_bioclinicalBERT_updated(train["AN_PROC_NAME"], name)
            X_test = get_bioclinicalBERT_updated(test["AN_PROC_NAME"], name)
                    
            with open(f"{name}.pickle", 'wb') as f:
                pickle.dump([X_train,X_test], f)
        # Prepare training and testing data

        y_train = train[outcome_col]
        y_test = test[outcome_col]

        if y_train.isna().any() or y_test.isna().any(): 
            na_indices = y_train.isna()  
            X_train = (X_train[~na_indices])
            y_train = (((y_train.dropna())).astype(int)).reset_index(drop=True)
            na_indices = y_test.isna()  
            X_test = X_test[~na_indices]  
            y_test = ((y_test.dropna()).astype(int)).reset_index(drop=True)
        else:
            pass
        
        model = RandomForestClassifier(random_state=42)
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='average_precision',verbose=2)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict_proba(X_test)[:, 1]

        # Compute the AUROC and AUPRC
        auroc = roc_auc_score(y_test, y_pred)
        auprc = average_precision_score(y_test, y_pred)

        # Save the scores
        auroc_scores.append(auroc)
        auprc_scores.append(auprc)
        with open(f"update_{outcome_col}_rf.pickle", 'wb') as f:
            pickle.dump({"auprc":auprc_scores, "auroc":auroc_scores}, f)

    # Now you can calculate the mean and standard deviation
    mean_auroc = np.mean(auroc_scores)
    std_auroc = np.std(auroc_scores)
    mean_auprc = np.mean(auprc_scores)
    std_auprc = np.std(auprc_scores)
    
    return(f"metrics: Mean AUROC: {mean_auroc}, SD AUROC: {std_auroc}, Mean AUPRC: {mean_auprc}, SD AUPRC: {std_auprc}")


## Death in 30

In [7]:
results=K_fold_val("death_in_30", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   1.0s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

1it [04:23, 263.77s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

2it [08:41, 260.43s/it]

...

3it [13:02, 260.48s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

4it [17:18, 258.64s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

5it [21:44, 260.92s/it]


In [8]:
print(results) 


    metrics: Mean AUROC: 0.87, SD AUROC: 0.01, 
    
 Mean AUPRC: 0.18, SD AUPRC: 0.02, 
    
 Mean ACC:0.98, SD ACC:0.0, 
    
 Mean precision:0.37, std precision:0.09, 
    
 mean recall:0.03, std_recall:0.01,
    
 mean specificity:1.0, std specificity:0.0, 
    
 mean f1:0.05, std f1:0.02 
    


### Logistic Regression

In [7]:
# implementing LR
results=K_fold_val_log_reg("death_in_30", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   2.8s
[CV] END ...................C=0.01, penal

[CV] END .........C=0.01, penalty=l2, solver=newton-cholesky; total time=   1.3s


In [8]:
print(results)

metrics: Mean AUROC: 0.8730819473081344, SD AUROC: 0.00989248205724818, Mean AUPRC: 0.18471069363510448, SD AUPRC: 0.021960429407393445


### Random Forest

In [ ]:
# implementing RF
results=K_fold_val_rf("death_in_30", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.2s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.2s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.2s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.2s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.2s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.2s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.6min
[CV] END .................max_depth=None, 

1it [38:53, 2333.55s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.4s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.2s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.2s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.2s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.3min
[CV] END .................max_depth=None, 

2it [1:15:56, 2268.70s/it]

...

3it [1:56:16, 2337.76s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.5s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.4s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.3s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.5min
[CV] END .................max_depth=None, 

4it [2:35:29, 2343.83s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.4s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.4s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.4s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.4s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.4s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.5s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.4s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.4s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.6min
[CV] END .................max_depth=None, 

5it [3:16:42, 2360.55s/it]


In [16]:
print(results)

metrics: Mean AUROC: 0.869983424761195, SD AUROC: 0.008341883435860854, Mean AUPRC: 0.1884577667226436, SD AUPRC: 0.022523583370812526


## Part II: DVT

In [9]:
results=K_fold_val("DVT", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
1it [03:55, 235.50s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2it [07:51, 235.60s/it]

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
3it [11:44, 234.60s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
4it [15:37, 233.79s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
5it [19:35, 235.00s/it]


In [10]:
print(results) 


    metrics: Mean AUROC: 0.78, SD AUROC: 0.02, 
    
 Mean AUPRC: 0.03, SD AUPRC: 0.01, 
    
 Mean ACC:0.99, SD ACC:0.0, 
    
 Mean precision:0.0, std precision:0.0, 
    
 mean recall:0.0, std_recall:0.0,
    
 mean specificity:1.0, std specificity:0.0, 
    
 mean f1:0.0, std f1:0.0 
    


### Logistic Regression

In [ ]:
# implementing LR
results=K_fold_val_log_reg("DVT", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   2.0s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=9.972e-09): result may not be accurate.
  warnings.warn(


[CV] END ............C=1, penalty=l2, solver=newton-cholesky; total time=   0.3s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=9.92328e-09): result may not be accurate.
  warnings.warn(


...

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=1.08184e-09): result may not be accurate.
  warnings.warn(


[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.2s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=1.03875e-09): result may not be accurate.
  warnings.warn(
/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
-------------------------------------------

[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.2s


5it [19:34, 234.89s/it]


In [18]:
print(results)

metrics: Mean AUROC: 0.7900274698489985, SD AUROC: 0.021959362963031324, Mean AUPRC: 0.027816162833981906, SD AUPRC: 0.007216474876399818


### Random Forest

In [ ]:
results=K_fold_val_rf("DVT", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.5s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.4s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.5s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.5s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.5s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.5s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.4s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.4s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.5s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.5min
[CV] END .................max_depth=None, 

1it [39:16, 2356.15s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.5s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.5s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.5s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.9min
[CV] END .................max_depth=None, 

2it [1:20:57, 2441.50s/it]

...

3it [2:03:40, 2497.15s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.5s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.5s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.9min
[CV] END .................max_depth=None, 

4it [2:44:09, 2470.23s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  30.0s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.4s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.5s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.9min
[CV] END .................max_depth=None, 

5it [3:27:07, 2485.43s/it]


In [20]:
print(results)

metrics: Mean AUROC: 0.7927061384997254, SD AUROC: 0.019533153653873873, Mean AUPRC: 0.027030845291581666, SD AUPRC: 0.007422983809221244


## Part III: PE

In [11]:
results=K_fold_val("PE", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
1it [03:47, 227.95s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2it [07:29, 224.31s/it]

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
3it [11:13, 224.28s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
4it [14:56, 223.59s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
5it [18:42, 224.46s/it]


In [12]:
print(results) 


    metrics: Mean AUROC: 0.74, SD AUROC: 0.02, 
    
 Mean AUPRC: 0.02, SD AUPRC: 0.01, 
    
 Mean ACC:1.0, SD ACC:0.0, 
    
 Mean precision:0.0, std precision:0.0, 
    
 mean recall:0.0, std_recall:0.0,
    
 mean specificity:1.0, std specificity:0.0, 
    
 mean f1:0.0, std f1:0.0 
    


### Logistic Regression

In [21]:
results=K_fold_val_log_reg("PE", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   1.8s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=9.96897e-09): result may not be accurate.
  warnings.warn(


[CV] END ............C=1, penalty=l2, solver=newton-cholesky; total time=   0.2s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=9.92831e-09): result may not be accurate.
  warnings.warn(


...

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=1.10332e-09): result may not be accurate.
  warnings.warn(


[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.2s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=1.03922e-09): result may not be accurate.
  warnings.warn(
/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
-------------------------------------------

[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.2s


5it [16:34, 198.97s/it]


In [22]:
print(results)

metrics: Mean AUROC: 0.7415067283885441, SD AUROC: 0.061810732693630736, Mean AUPRC: 0.01816573624180773, SD AUPRC: 0.011264951955630605


### Random Forest

In [23]:
results=K_fold_val_rf("PE", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.7s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.1min
[CV] END .................max_depth=None, 

1it [37:07, 2227.64s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.9s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.9s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.9s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.9s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.7s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.4min
[CV] END .................max_depth=None, 

2it [1:14:49, 2247.59s/it]

...

3it [1:54:36, 2311.23s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  41.0s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  58.8s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  30.1s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.8s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.1min
[CV] END .................max_depth=None, 

4it [2:30:49, 2256.71s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.9s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  30.1s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  30.0s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.9s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  43.2s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  53.8s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  30.0s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.9s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.9s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.3min
[CV] END .................max_depth=None, 

5it [3:11:14, 2294.96s/it]


In [24]:
print(results)

metrics: Mean AUROC: 0.7557299565733724, SD AUROC: 0.03425466319371343, Mean AUPRC: 0.012509759479323554, SD AUPRC: 0.002630535724167932


## Part 4: PNA

In [13]:
results=K_fold_val("PNA", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
1it [03:56, 236.85s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
2it [07:50, 234.70s/it]

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
3it [11:43, 234.12s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.7s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
4it [15:34, 232.96s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

/home/charles/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
5it [19:33, 234.66s/it]


In [14]:
print(results) 


    metrics: Mean AUROC: 0.84, SD AUROC: 0.02, 
    
 Mean AUPRC: 0.05, SD AUPRC: 0.02, 
    
 Mean ACC:0.99, SD ACC:0.0, 
    
 Mean precision:0.0, std precision:0.0, 
    
 mean recall:0.0, std_recall:0.0,
    
 mean specificity:1.0, std specificity:0.0, 
    
 mean f1:0.0, std f1:0.0 
    


### Logistic Regression

In [25]:
results=K_fold_val_log_reg("PNA", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   2.8s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=9.96897e-09): result may not be accurate.
  warnings.warn(


[CV] END ............C=1, penalty=l2, solver=newton-cholesky; total time=   0.3s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=9.93514e-09): result may not be accurate.
  warnings.warn(


...

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=1.11946e-09): result may not be accurate.
  warnings.warn(


[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.3s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=1.06513e-09): result may not be accurate.
  warnings.warn(
/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
-------------------------------------------

[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.3s


5it [19:02, 228.53s/it]


In [26]:
print(results)

metrics: Mean AUROC: 0.8499443712795675, SD AUROC: 0.019036823463044007, Mean AUPRC: 0.05724634702342416, SD AUPRC: 0.019860223427260803


### Random Forest

In [27]:
results=K_fold_val_rf("PNA", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.7s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.2min
[CV] END .................max_depth=None, 

1it [36:36, 2196.00s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.7s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.3min
[CV] END .................max_depth=None, 

2it [1:14:08, 2229.37s/it]

...

3it [1:52:01, 2249.19s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.5s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.6s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.4min
[CV] END .................max_depth=None, 

4it [2:31:33, 2297.86s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.8s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  29.8s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 3.7min
[CV] END .................max_depth=None, 

5it [3:14:27, 2333.42s/it]


In [28]:
print(results)

metrics: Mean AUROC: 0.8382779935868185, SD AUROC: 0.021394571402566037, Mean AUPRC: 0.054881186567758866, SD AUPRC: 0.015436462736670587


## Part 5: post_AKI_status

In [15]:
results=K_fold_val("post_aki_status", df) 

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

1it [04:58, 298.73s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

2it [09:49, 293.97s/it]

...

3it [14:39, 292.07s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

4it [19:30, 291.70s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.8s
[CV] END .learning_rate=0.1, max_depth=4

5it [24:26, 293.32s/it]


In [16]:
print(results) 


    metrics: Mean AUROC: 0.84, SD AUROC: 0.0, 
    
 Mean AUPRC: 0.5, SD AUPRC: 0.01, 
    
 Mean ACC:0.88, SD ACC:0.0, 
    
 Mean precision:0.67, std precision:0.01, 
    
 mean recall:0.24, std_recall:0.01,
    
 mean specificity:0.98, std specificity:0.0, 
    
 mean f1:0.35, std f1:0.01 
    


### Logistic Regression

In [29]:
results=K_fold_val_log_reg("post_aki_status", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   5.9s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=9.97338e-09): result may not be accurate.
  warnings.warn(


[CV] END ............C=1, penalty=l2, solver=newton-cholesky; total time=   0.3s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=9.93386e-09): result may not be accurate.
  warnings.warn(


...

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=1.11028e-09): result may not be accurate.
  warnings.warn(


[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.3s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=1.03238e-09): result may not be accurate.
  warnings.warn(
/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
-------------------------------------------

[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.3s


5it [18:41, 224.31s/it]


In [30]:
print(results)

metrics: Mean AUROC: 0.8444442750861644, SD AUROC: 0.0020515831399419964, Mean AUPRC: 0.4986129666907738, SD AUPRC: 0.008742264974877519


### Random Forest

In [31]:
results=K_fold_val_rf("post_aki_status", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.2s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.2s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.2s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.2s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.2s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.2s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 2.5min
[CV] END .................max_depth=None, 

1it [29:42, 1782.75s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.4s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.4s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.3s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 2.4min
[CV] END .................max_depth=None, 

2it [58:07, 1736.81s/it]

...

3it [1:27:11, 1740.31s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.2s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.2s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.2s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.3s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 2.6min
[CV] END .................max_depth=None, 

4it [1:56:22, 1744.58s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.2s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.4s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.2s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.3s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=  28.3s
[CV] END .................max_depth=None, min_samples_leaf=1; total time= 2.5min
[CV] END .................max_depth=None, 

5it [2:25:13, 1742.77s/it]


In [32]:
print(results)

metrics: Mean AUROC: 0.8407842180125844, SD AUROC: 0.0015965445840693702, Mean AUPRC: 0.49019568833455107, SD AUPRC: 0.006167623360268969


## delirium

In [17]:
results=K_fold_val("postop_del", df)


0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4

1it [02:30, 150.34s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4

2it [04:59, 149.40s/it]

...

3it [07:26, 148.69s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4

4it [09:55, 148.73s/it]

passed
Fitting 5 folds for each of 45 candidates, totalling 225 fits
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=1; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=2; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4, min_child_weight=4; total time=   0.3s
[CV] END .learning_rate=0.1, max_depth=4

5it [12:24, 148.97s/it]


In [18]:
print(results) 


    metrics: Mean AUROC: 0.7, SD AUROC: 0.01, 
    
 Mean AUPRC: 0.67, SD AUPRC: 0.02, 
    
 Mean ACC:0.65, SD ACC:0.01, 
    
 Mean precision:0.63, std precision:0.02, 
    
 mean recall:0.59, std_recall:0.02,
    
 mean specificity:0.7, std specificity:0.02, 
    
 mean f1:0.61, std f1:0.02 
    


### Logistic Regression

In [33]:
results=K_fold_val_log_reg("postop_del", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END ...................C=0.01, penalty=l1, solver=lbfgs; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END .........C=0.01, penalty=l1, solver=newton-cholesky; total time=   0.0s
[CV] END ...................C=0.01, penalty=l2, solver=lbfgs; total time=   0.2s
[CV] END ...................C=0.01, penal

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=2.38448e-08): result may not be accurate.
  warnings.warn(
/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features o

[CV] END ............C=1, penalty=l2, solver=newton-cholesky; total time=   0.1s
[CV] END ............C=1, penalty=l2, solver=newton-cholesky; total time=   0.1s
[CV] END ............C=1, penalty=l2, solver=newton-cholesky; total time=   0.1s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=2.38218e-08): result may not be accurate.
  warnings.warn(
/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features o

...

/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.65123e-09): result may not be accurate.
  warnings.warn(
/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features o

[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.1s
[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.1s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.65843e-09): result may not be accurate.
  warnings.warn(
/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features o

[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.1s
[CV] END ...........C=10, penalty=l2, solver=newton-cholesky; total time=   0.1s


/home/charles/.local/lib/python3.8/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=4.65296e-09): result may not be accurate.
  warnings.warn(
/home/charles/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
-------------------------------------------

In [34]:
print(results)

metrics: Mean AUROC: 0.6986003109895802, SD AUROC: 0.016179263613710124, Mean AUPRC: 0.6753648691759397, SD AUPRC: 0.02404128529050899


### Random Forest

In [35]:
results=K_fold_val_rf("postop_del", df)

0it [00:00, ?it/s]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END .................max_depth=None, min_samples_leaf=1; total time=  10.9s
[CV] END .................max_depth=None, 

1it [02:23, 143.21s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END .................max_depth=None, min_samples_leaf=1; total time=  11.0s
[CV] END .................max_depth=None, 

2it [04:47, 143.88s/it]

...

3it [07:13, 144.75s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END .................max_depth=None, min_samples_leaf=1; total time=  11.1s
[CV] END .................max_depth=None, 

4it [09:38, 144.96s/it]

passed
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.6s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.6s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.6s
[CV] END ....................max_depth=4, min_samples_leaf=1; total time=   3.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.7s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.6s
[CV] END ....................max_depth=4, min_samples_leaf=3; total time=   3.6s
[CV] END .................max_depth=None, min_samples_leaf=1; total time=  11.2s
[CV] END .................max_depth=None, 

5it [12:04, 144.81s/it]


In [36]:
print(results)

metrics: Mean AUROC: 0.7022637252485312, SD AUROC: 0.012780494844257194, Mean AUPRC: 0.680858101593589, SD AUPRC: 0.02080692117216303
